# BART DEMO

## 0. BART_base Architecture Overview

In [ ]:
# Load the model
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base', add_prefix_space=True)
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
print(bart_model)

## 1. Mask infilling

In [ ]:
from transformers import AutoTokenizer, BartForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

text = "The <mask> was stuck in the tree"
input_ids = tokenizer([text], return_tensors="pt")["input_ids"]
logits = model(input_ids).logits

masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item() #extract mask index
probs = logits[0, masked_index].softmax(dim=0) #convert into probability
values, predictions = probs.topk(5) #take top 5 predictions

tokenizer.decode(predictions).split()

## 2. News summerization

In [2]:
from transformers import AutoTokenizer, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

# text = (
#     "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
#     "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
#     "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
# )

text = (
  "The Justice Department's recommendation comes after a federal judge ruled in August "
  "that Google had violated US antitrust law with its search business. The ruling, "
  "in which the judge called Google a 'monopolist', set the stage for changes to "
  "Google's oldest and most important business and for how millions of Americans get information online"
)

inputs = tokenizer([text], max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=30)
result = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(result)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


The Justice Department's recommendation comes after a federal judge ruled in August that Google had violated US antitrust law with its search business. The ruling


## 3. Sentiment Analysis (Sequence Classification)

Bart model with a sequence classification/head on top (a linear layer on top of the pooled output) e.g. for GLUE tasks.

In [4]:
import torch
from transformers import AutoTokenizer, BartForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("valhalla/bart-large-sst2")
model = BartForSequenceClassification.from_pretrained("valhalla/bart-large-sst2")

# 2-star review (out of 10)
# text = "It was a complete waste of time. As a massive fan of Joker, I expected a strong comeback five years later. The movie was a complete drag. Whenever I thought the movie was taking a turn for the better, it got worse. Save yourself the time and money. Joaquin Phoenix's performance was excellent, but the script was terrible. Through no fault of his own, Joker Folie A Deux is in line as one of the worst sequels ever. The movie seemed more like a display of Lady Gaga's singing and acting ability, which isn't great. I believe this may be the biggest box office flop of 2024. This movie should have been released on Tubi for free."

# 8-star review
text = "The acting by Phoenix and Gaga is top tier. Whilst people currently dislike it due to it having an alternate musical genre people will realise the masterpiece that has been created using an alternate genre not following the same niche that has been the meta for the previous decade. The songs are perfectly spaced adding an enjoyable element to continue to register the audiences engagement throughout the film. Furthermore the movie offers and amazing deep dive into insanity and a split personality whilst offering an in depth and entertaining court room scene that offers the perfect mix of drama, comedy and suspense."

inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


'POSITIVE'